## **Currency Conversion Tool.**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [ ]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency:str) -> float:
    '''
    This functions fetches the currency conversion factor between a given base currency and a target currency.
    '''
    url = f'https://v6.exchangerate-api.com/v6/fe477a9b25f40c4231d0873d/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()



@tool
def convert(base_currency_val: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    '''
    Given a currncy conversion rate to this function calculates the target currency value from a givem base currency value.
    '''
    return base_currency_val *  conversion_rate

In [ ]:
get_conversion_factor.invoke({'base_currency': 'USD', 'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1755561601,
 'time_last_update_utc': 'Tue, 19 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1755648001,
 'time_next_update_utc': 'Wed, 20 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.3868}

In [ ]:
convert.invoke({'base_currency_val': 11.9, 'conversion_rate':87.3868})

1039.90292

tool binding

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm_with_tool = model.bind_tools([get_conversion_factor, convert])

In [ ]:
message =  [HumanMessage("What's the conversion rate between USD and INR , and based on that can you convert 10 usd to inr.")]

In [ ]:
message

[HumanMessage(content="What's the conversion rate between USD and INR , and based on that can you convert 10 usd to inr.", additional_kwargs={}, response_metadata={})]

In [ ]:
llm_with_tool.invoke(message).tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '1f3e90cb-68f9-48d3-849f-d668288d5cf5',
  'type': 'tool_call'}]

In [ ]:
ai_msg = llm_with_tool.invoke(message)

In [ ]:
message.append(ai_msg)

In [ ]:
ai_msg.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '79556fad-bddf-48f7-aed5-783d34b71ec4',
  'type': 'tool_call'}]

In [ ]:
import json

for tool_call in ai_msg.tool_calls:
    if tool_call['name'] == 'get_conversion_factor':
        tool_msg1  = get_conversion_factor.invoke(tool_call)

        conversion_rate = json.loads(tool_msg1.content)['conversion_rate']

        message.append(tool_msg1)

    if tool_call['name'] == 'convert':
        tool_call['args']['conversion_rate'] = conversion_rate


        tool_msg2  = convert.invoke(tool_call)
        
        message.append(tool_msg2)
    

In [ ]:
message

[HumanMessage(content="What's the conversion rate between USD and INR , and based on that can you convert 10 usd to inr.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--93600c5d-5546-4d23-b786-7160e34a9a31-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '79556fad-bddf-48f7-aed5-783d34b71ec4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 100, 'output_tokens': 13, 'total_tokens': 113, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms",

In [125]:
llm_with_tool.invoke(message)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 29
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing de

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 25
}
]

In [ ]:
llm_with_tool.invoke(message).content

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 20
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing de

''